In [2]:
using LinearAlgebra
using LatticeQM

see PRB 96, 075311 (2017): Lattice relaxation and energy band modulation in twisted bilayer graphene

In [23]:
lat = Geometries2D.graphene()
S = det(Structure.getA(lat)); # unit cell area
b1 = Structure.getB(lat)[:,1]
b2 = Structure.getB(lat)[:,2];

In [16]:
° = π/180
θ = 1.1°

0.019198621771937627

In [24]:
using LatticeQM.Utils: rotation2D
id = Matrix(1.0*I, 2,2)
G1 = (id - rotation2D(θ)) * b1
G2 = (id - rotation2D(θ)) * b2;

In [26]:
Δϵ = 0.0189 #eV
V0 = (4/9)*Δϵ/S
λ = 3.5 # eV / Angstrom^2
μ = 7.8; # eV / Angstrom^2

We are looking for the relative displacement field $\vec{u}(\vec{r})$ such that

\begin{align}
\vec{\delta}(\vec{r}) = \vec{\delta}_0(\vec{r}) + \vec{u}(\vec{r}) \; .
\end{align}

\begin{align}
u(\vec{r}) = u_1(\vec{r})\;\vec{a}_1+ u_2(\vec{r})\;\vec{a}_2 = \sum_{\vec{G}} \left(u^{(1)}_{\vec{G}}\; \vec{a}_1+u^{(2)}_{\vec{G}}\; \vec{a}_2\right) \; e^{i \vec{G}\cdot\vec{r}}
\end{align}

\begin{align}
\sin\left[\vec{G}_j\cdot\vec{r}+\vec{b}_j\cdot \vec{u}(\vec{r})\right] = \sum_{\vec{G}} f^j_{\vec{G}} \; e^{i\vec{G}\cdot\vec{r}}
\end{align}

With $\vec{r}=x_1 \vec{L}_1+x_2 \vec{L}_2$ and $\vec{a}_i\cdot\vec{b}_j = 2\pi \delta_{ij}$ and $\vec{L}_i\cdot\vec{G}_j = 2\pi \delta_{ij}$ and $\vec{G}=n_1\vec{G}_1+n_2\vec{G}_2$ we get

\begin{align}
\sin\left[x_j+u_j(\vec{x})\right] = \sum_{n_1n_2} f^j_{n_1n_2} \; e^{i 2\pi (n_1 x_1 + n_2 x_2)}
\end{align}

\begin{align}
\vec{u}_{\vec{G}} = \sum_{j=1}^{3} 4 V_0 f_{\vec{G}}^j K_{\vec{G}}^{-1} \vec{b}_j
\end{align}

\begin{align}
u^{(i)}_{\vec{G}} = \sum_{j=1}^{3} 4 V_0 f_{\vec{G}}^j \cdot \, (\vec{a}_i\,K_{\vec{G}}^{-1} \vec{b}_j)
\end{align}

\begin{align}
K_{\vec{G}} = 
\end{align}

In [20]:
basisG = [[i,j] for i=-10:10 for j=-10:10];

In [18]:
function Kinv(q; B=Matrix(1.0I,2,2), λ, μ)
    G = B * q
    
    d = 1/(sum(abs2,G)^2 * μ * (λ+2μ))
    f1 = λ+2μ; f2 = λ+μ
    d * [G[1]^2*μ + G[2]*f1  -G[1]*G[2]*f2;
        -G[1]*G[2]*f2        G[2]^2*μ+G[1]^2*f1]
end

Kinv (generic function with 1 method)

In [ ]:
function fG(uGs, basis)
    
    Gs = [[1,0],[0,1],[-1,-1]]
    
    u(x::AbstractVector) = sum(uG*exp(1.0im*G*x) for (uG,G) in zip(uGs,basis))
    
    f(x::AbstractVector) = sin(x + )
    
end

In [ ]:
function update!(u2, u1, basisG; B=Matrix(1.0I,2,2), V0, λ, μ)
    
    bs = [b1, b2, b2-b1]
    Gs = [G1, G2, G2-G1]
    
    fGs = fG(u1, bs, Gs, basis)
    
    for (i,q) in enumerate(basisG)
        Kinv0 = Kinv(q; B=B, λ=λ, μ=μ)
        for (j,b) in enumerate(bs)
            u2[i] += 4*V0*fGs[i,j]*Kinv0*b
        end
    end
    
    
end